# Preparacion de los datos

Esta sección veremos las operaciones más frecuentes dar formato (estandarizar) a los datos para su posterior tratamiento, dichas operaciones usualmente son:

* Transformación de campos
* Eliminación de datos duplicados
* Eliminación o relleno de datos faltantes
* Reducción de atributos

Ahora sin mas preambulos vamos a tomar la base de datos <a href="https://archive.ics.uci.edu/ml/datasets/Adult" target="_blank">Adult Data Set</a> la cual contiene un total de **14** columnas (atributos, o características) y **48842** registros.


En el siguiente fragmento de código vamos a aprender a instalar librerias (en caso que este ejemplo se este ejecutando en un ambiente local es bastante útil), e instalaremos:

* La libreria para el análisis y manipulación de datos **Pandas** 
* La libreria para hacer peticiones http **requests** y la necesitamos para acceder a la base de datos que esta disponible en https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data 

In [1]:
!pip install pandas
!pip install requests

En el siguiente fragmento de código veremos como se importa la libreria **pandas** y agregaremos una configuración para la previsualización de la base de datos; puesto que en caso de no agregar dicha configuración en al hacer la previsualización nos mostraria todos los registros y dado que son **48842** tardaria un tiempo considerable en cargar.

In [2]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

Ahora vamos a importar la libreria **requests** las cuales usaremos para resolver la peticion **HTTP** y obtener la base de datos en un stream.

In [3]:

import requests

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
data_stream = requests.get(url).content

Ahora se importa la libreria **io** para transformar el stream que trae la base de datos en un formato compatible con un dataframe de pandas

Adicionalmente hay que ajustar el formato en el que este la base de datos en ocasiones agregando manualmente ciertas configuraciones para poder leer la base de datos correctamente ya que por defecto la funcion *pd.read_csv* asume que el archivo esta bien formateado incluyendo las cabeceras y que el separador entre atributos sea ","

En este caso la base de datos no trae implicitos los nombres de las columnas (cabeceras) por lo tanto se lo vamos a indicar manualmente, y toca modificar el separador para que sea ", " reemplazando el que trae por defecto con valor "," 

In [4]:
import io

headers = ['age', 'work_class', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'salary']
dataset=pd.read_csv(io.StringIO(data_stream.decode('utf-8')), header=None, names = headers, delimiter = ", ")
dataset

<ipython-input-4-043368e12134>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset=pd.read_csv(io.StringIO(data_stream.decode('utf-8')), header=None, names = headers, delimiter = ", ")


,age,work_class,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


Antes de continuar, es necesario tener en cuenta los diferentes tipos de valores a nivel estadístico ya que facilitan la comprensión de los datos.

* **Valores numéricos**,
    * **Discretos**, se evidencian cuando se puede asumir (determinar) el conjunto finito de valores que puede tomar una variable númerica. Por ejemplo, el rango de valores de la edad en años 0, 1, 2, 3....150.
    * **Continuos**, muy usados cuando se hacen mediciones y no se puede establecer facilmente de forma determinista el rango de posibles valores. Por ejemplo, podríamos encontrar el valor de una divisa, si se tuvieran registros de la temperatura de un lugar.
* **Valores categoricos**, son valores asociados a alguna característica y en términos del análisis de datos se interpretan como variables discretos
    * **Nominales** no aportan un valor cualitativo, y son usados únicamente para efectos de clasificación. Por ejemplo la lista de paises Colombia, Ecuador, Peru, Venezuela...
    * **Ordinales** aportan un valor cuantitativo sujeto a la interpretación o intención del análisis, dado que se les da algún tipo de sentido u orden. Por ejemplo las clases sociales en terminos de ingresos Baja, Media, Alta, Muy Alta 


Ahora vamos a revisar un poco mas detenidamente la descripción de los datos que vienen en la documentación:

* age: continuous.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* fnlwgt: continuous.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuous.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: continuous.
* capital-loss: continuous.
* hours-per-week: continuous.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
* salary: >50K, <=50K.


Teniendo en cuenta que la gran mayoria de algoritmos utilizan variables númericas (no nulas) vamos a pensar en alguna transformación para cada campo.

| Campo | Tipo de valor | Tiene valores nulos | Transformación |
|:-:|:-:|:-:|:-|
| age            |  númerico - discreto  | no | no aplica |
| workclass      |  categórico - nominal | si | se va a mapear cada elemento a un valor único |
| fnlwgt         |  númerico - continuo  | no | no aplica |
| education      |  categórico - ordinal | si | se va a mapear cada elemento a un valor único númerico ascendente |
| education-num  |  númerico - discreto  | no | no aplica |
| marital-status |  categorico - nominal | si | se va a mapear cada elemento a un valor único númerico |
| occupation     |  categórico - nominal | si | se va a mapear cada elemento a un valor único númerico |
| relationship   |  categórico - nominal | si | se va a mapear cada elemento a un valor único númerico |
| race           |  categórico - nominal | si | se va a mapear cada elemento a un valor único númerico |
| sex            |  categórico - nominal | si | se va a mapear cada elemento a un valor único númerico |
| capital-gain   |  númerico - continuo  | no | no aplica |
| capital-loss   |  númerico - continuo  | no | no aplica |
| hours-per-week |  númerico - discreto  | no | no aplica |
| native-country |  categórico - nominal | si | se va a mapear cada elemento a un valor único númerico |
| salary         |  categórico - nominal | si | se va a mapear cada elemento a un valor único númerico |



**Observación.** Hay campos en los que se podría pensar en algún tipo de agrupamiento o clasificación, por ejemplo:

* En el campo **"age"** podría pensarse en una reducción de categorías en el sentido que los valores [15, 16, 17, 18, 19, 20] se agruparia a la categoria adolescentes con valor "1", otro grupo para adultos con valor "2", adultos mayores con valor "3" y viejitos con valor "4". Esto a lo mejor nos da un mejor rendimiento en la ejecucion de los algoritmos puesto que reduce los valores posibles, pero en el peor caso podría arrojar como resultado modelos ambiguos y poco confiables.
* En el campo **"education"** así por encima se podría tomar como "categórico - nominal" pero si se le establece un orden a las categorias, por ejemplo el grado de escolaridad de menor a mayor lo convertiriamos en un valor "categórico - ordinal" ya que le estamos dando un sentido a los datos.

A continuación tenemos la definición de los mapeos que realizaremos de forma explícita a los diferentes campos mencionados en la tabla anterior; pero ahora tenemos el reto de utilizar algun tipo de estrategia para tratar los datos nulos, por lo tanto es pertinente revisar las siguientes estrategias ordenadas de menor a mayor complejidad:

1. Eliminar los registros completos que contengan datos nulos.
2. Utilizar alguna estrategia para completar los datos.
    * Asignar un valor por defecto
    * En caso de tener variables categoricas o discretas utilizar alguna medida de tendencia central como lo puede ser la moda (el dato que mas se repite)
    * En caso de variables continuas, utilizar alguna medida de tendencia central como lo puede ser la moda o el promedio.

**Nota.** En ocasiones esas estrategias para rellenar los datos faltantes pueden no ser tan efectivas para el funcionamiento de los modelos, pero si el investigador lo considera pertinente podría odenar los atributos de mayor a menor peso, agrupar los datos y tomar alguna medida de tendencia central con los datos agrupados. Como se que esto es un poco enredado, podríamos pensar en el siguiente ejemplo:

En una base de datos con los campos "Edad", "Profesion", "Salario" donde el campo "Salario" posee datos nulos, una primera aproximación podria ser usar el promedio, o la moda para rellenar esos datos faltantes; pero si la intención es ser un poco mas detallado y preciso podríamos definir los datos faltantes del campo "Salario" como el promedio o la moda de todos los campos salarios agrupados por edad, o profesion, o ambos.

**Observación.** Para efectos del presente tutorial vamos a utilizar la estrategia de asignar un valor por defecto a esos datos nulos.


In [5]:
# Mapping functions

work_class_mapper = {
    "Never-worked": 1,
    "Without-pay": 2,
    "State-gov": 3,
    "Local-gov": 4,
    "Federal-gov": 5,
    "Self-emp-inc": 6,
    "Self-emp-not-inc": 7,
    "Private": 8,
    "None": 9,
    "?": 0
}
def transform_work_class(x):
    return work_class_mapper[x]

education_mapper = {
    "Preschool": 1,
    "1st-4th": 2,
    "5th-6th": 3,
    "7th-8th": 4,
    "9th": 5,
    "10th": 6,
    "11th": 7,
    "12th": 8,
    "HS-grad": 9,
    "Bachelors": 10,
    "Assoc-acdm": 11,
    "Assoc-voc": 12,
    "Some-college": 13,
    "Prof-school": 14,
    "Masters": 15,
    "Doctorate": 16,
    "?": 0
}
def transform_education(x):
    return education_mapper[x]

marital_status_mapper = {
    "Married-civ-spouse": 1,
    "Divorced": 2,
    "Never-married": 3,
    "Separated": 4,
    "Widowed": 5,
    "Married-spouse-absent": 6,
    "Married-AF-spouse": 7,
    "?": 0
}
def transform_marital_status(x):
    return marital_status_mapper[x]

occupation_mapper = {
    "Tech-support": 1,
    "Craft-repair": 2,
    "Other-service": 3,
    "Sales": 4,
    "Exec-managerial": 5,
    "Prof-specialty": 6,
    "Handlers-cleaners": 7,
    "Machine-op-inspct": 8,
    "Adm-clerical": 9,
    "Farming-fishing": 10,
    "Transport-moving": 11,
    "Priv-house-serv": 12,
    "Protective-serv": 13,
    "Armed-Forces": 14,
    "?": 0
}
def transform_occupation(x):
    return occupation_mapper[x]

relationship_mapper = {
    "Wife": 1,
    "Own-child": 2,
    "Husband": 3,
    "Not-in-family": 4,
    "Other-relative": 5,
    "Unmarried": 6,
    "?": 0
}
def transform_relationship(x):
    return relationship_mapper[x]

race_mapper = {
    "White": 1,
    "Asian-Pac-Islander": 2,
    "Amer-Indian-Eskimo": 3,
    "Other": 4,
    "Black": 5,
    "?": 0
}
def transform_race(x):
    return race_mapper[x]

sex_mapper = {
    "Male": 1,
    "Female": 2,
    "?": 0
}
def transform_sex(x):
    return sex_mapper[x]

country_mapper = {
    "United-States": 1, 
    "Cambodia": 2, 
    "England": 3, 
    "Puerto-Rico": 4, 
    "Canada": 5, 
    "Germany": 6, 
    "Outlying-US(Guam-USVI-etc)": 7, 
    "India": 8, 
    "Japan": 9, 
    "Greece": 10, 
    "South": 11, 
    "China": 12,
    "Cuba": 13, 
    "Iran": 14, 
    "Honduras": 15, 
    "Philippines": 16, 
    "Italy": 17, 
    "Poland": 18, 
    "Jamaica": 19, 
    "Vietnam": 20, 
    "Mexico": 21, 
    "Portugal": 22, 
    "Ireland": 23, 
    "France": 24, 
    "Dominican-Republic": 25, 
    "Laos": 26, 
    "Ecuador": 27, 
    "Taiwan": 28, 
    "Haiti": 29, 
    "Columbia": 30, 
    "Hungary": 31, 
    "Guatemala": 32, 
    "Nicaragua": 33, 
    "Scotland": 34, 
    "Thailand": 35, 
    "Yugoslavia": 36, 
    "El-Salvador": 37, 
    "Trinadad&Tobago": 38, 
    "Peru": 39, 
    "Hong": 40, 
    "Holand-Netherlands": 41,
    "?": 0
}
def transform_country(x):
    return country_mapper[x]

salary_mapper = {
    ">50K": 1,
    "<=50K": 2,
    ">50K.": 1,
    "<=50K.": 2,
    "?": 0
}
def transform_salary(x):
    return salary_mapper[x]

En el siguiente fragmento de código se puede ver como se aplican estas transformaciones a los datos; para no afectar el conjunto de datos inicial definiendo nuevos campos con el sufijo **"_transformed"** con el formato adecuado para el tratamiento de los algoritmos de ML

In [6]:
# The following lines contains the mapping configurations for ML proposals
dataset['work_class_transformed'] = dataset['work_class'].apply(lambda x: transform_work_class(x))
dataset['education_transformed'] = dataset['education'].apply(lambda x: transform_education(x))
dataset['marital_status_transformed'] = dataset['marital_status'].apply(lambda x: transform_marital_status(x))
dataset['occupation_transformed'] = dataset['occupation'].apply(lambda x: transform_occupation(x))
dataset['relationship_transformed'] = dataset['relationship'].apply(lambda x: transform_relationship(x))
dataset['race_transformed'] = dataset['race'].apply(lambda x: transform_race(x))
dataset['sex_transformed'] = dataset['sex'].apply(lambda x: transform_sex(x))
dataset['native_country_transformed'] = dataset['native_country'].apply(lambda x: transform_country(x))
dataset['salary_transformed'] = dataset['salary'].apply(lambda x: transform_salary(x))

dataset

,age,work_class,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary,work_class_transformed,education_transformed,marital_status_transformed,occupation_transformed,relationship_transformed,race_transformed,sex_transformed,native_country_transformed,salary_transformed
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,3,10,3,9,4,1,1,1,2
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,7,10,1,5,3,1,1,1,2
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,8,9,2,7,4,1,1,1,2
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,8,7,1,7,3,5,1,1,2
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,8,10,1,6,1,5,2,13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,8,11,1,1,1,1,2,1,2
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,8,9,1,8,3,1,1,1,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,8,9,5,9,6,1,2,1,2
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,8,9,3,9,2,1,1,1,2
